# 전동킥보드 주차장 위치
- 강남구
- 강서구
- 동작구
- 마포구
- 서대문구

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import pandas as pd
import requests, json


def get_location(address):
    url = "https://dapi.kakao.com/v2/local/search/address.json?query=" + address
    headers = {"Authorization": "KakaoAK REST API Key"}
    api_json = json.loads(str(requests.get(url, headers=headers).text))
    address = api_json["documents"][0]["address"]
    return float(address["y"]), float(address["x"]), address["address_name"]


def get_location_by_keyword_subway(keyword):
    url = "https://dapi.kakao.com/v2/local/search/keyword.json?query=" + keyword
    headers = {"Authorization": "KakaoAK REST API Key"}
    api_json = json.loads(str(requests.get(url, headers=headers).text))
    for document in api_json["documents"]:
        if document["category_name"] == "교통,수송 > 지하철,전철 > 지하철출구":
            return float(document["y"]), float(document["x"]), document["address_name"]


def get_location_by_keyword(keyword):
    url = "https://dapi.kakao.com/v2/local/search/keyword.json?query=" + keyword
    headers = {"Authorization": "KakaoAK REST API Key"}
    api_json = json.loads(str(requests.get(url, headers=headers).text))
    document = api_json["documents"][0]
    return float(document["y"]), float(document["x"]), document["address_name"]


path = "/content/drive/MyDrive/multicampus/data/"

In [5]:
col_list = ["Gu", "Latitude", "Longitude", "Address"]
parkingzone = pd.DataFrame(columns=col_list)

In [6]:
gangnam = pd.read_csv(path + "서울특별시 강남구_전동킥보드 주차구역_20221031.csv", encoding="cp949")
len(gangnam)

# for address in gangnam["도로명 주소"]:
#     lat, lng, addr = get_location(address)
#     gu = "강남구"
#     parkingzone = pd.concat([parkingzone, pd.DataFrame([[gu, lat, lng, addr]], columns=col_list)], ignore_index=True)

6

In [7]:
gangseo = pd.read_csv(path + "서울특별시 강서구_서울특별시 강서구_전동킥보드 주차구역_20221028.csv", encoding="cp949")
len(gangseo)
# for address in gangseo["전동킥보드 주차구역 주소"]:
#     lat, lng, addr = get_location(address)
#     gu = "강서구"
#     parkingzone = pd.concat([parkingzone, pd.DataFrame([[gu, lat, lng, addr]], columns=col_list)], ignore_index=True)

2

In [8]:
dongjak = pd.read_csv(path + "서울특별시 동작구_공유형 전동킥보드 주차구역_20230104.csv", encoding="cp949")
len(dongjak)

# for address in dongjak["상세주소"]:
#     lat, lng, addr = get_location("서울특별시 동작구 " + address)
#     gu = "동작구"
#     parkingzone = pd.concat([parkingzone, pd.DataFrame([[gu, lat, lng, addr]], columns=col_list)], ignore_index=True)

4

In [9]:
mapo = pd.read_excel(path + "서울특별시_마포구_전동킥보드 주차 위치_20230120.xlsx")
len(mapo)
# for address in mapo.loc[:9, "행정동"]:
#     lat, lng, addr = get_location(address)
#     gu = "마포구"
#     parkingzone = pd.concat([parkingzone, pd.DataFrame([[gu, lat, lng, addr]], columns=col_list)], ignore_index=True)

32

In [ ]:
for keyword in mapo.loc[10:20, "위치"]:
    lat, lng, addr = get_location_by_keyword_subway(keyword)
    gu = "마포구"
    parkingzone = pd.concat([parkingzone, pd.DataFrame([[gu, lat, lng, addr]], columns=col_list)], ignore_index=True)

for keyword in ["dmc역 8번", "dmc역 9번"]:
    lat, lng, addr = get_location_by_keyword_subway(keyword)
    gu = "마포구"
    parkingzone = pd.concat([parkingzone, pd.DataFrame([[gu, lat, lng, addr]], columns=col_list)], ignore_index=True)

In [ ]:
for keyword in mapo.loc[24:, "위치"]:
    lat, lng, addr = get_location_by_keyword(keyword)
    gu = "마포구"
    parkingzone = pd.concat([parkingzone, pd.DataFrame([[gu, lat, lng, addr]], columns=col_list)], ignore_index=True)

In [10]:
seodaemun = pd.read_csv(path + "서울특별시 서대문구_서대문구 전동킥보드 주차구역_20221025.csv", encoding="cp949")
len(seodaemun)
# lat, lng, addr = get_location_by_keyword_subway(seodaemun["위치"][0])
# gu = "서대문구"
# parkingzone = pd.concat([parkingzone, pd.DataFrame([[gu, lat, lng, addr]], columns=col_list)], ignore_index=True)

1

In [ ]:
parkingzone.reset_index(inplace=True)

In [ ]:
parkingzone.to_csv(path + "parkingzone.csv", index=False)

# 위도/경도 기준으로 거리 구하기? 가까운 곳 구하기?

In [ ]:
import pandas as pd
path = "/content/drive/MyDrive/multicampus/data/"

parkingzone = pd.read_csv(path + "parkingzone.csv")
parkingzone = parkingzone[parkingzone["Gu"]=="강남구"][["index", "Latitude", "Longitude"]]
parkingzone.to_csv(path + "parkingzone4.csv", index=False, encoding="utf-8")

In [ ]:
import requests
import json

time = 1000000000000
# nearestX = 0
# nearestY = 0
# address = ""

url = f"https://apis.openapi.sk.com/tmap/routes/pedestrian?version=1"

headers = {
    "appkey": "l7xx0ffb87c22bdb45ae8facaeeb7958ad36 ",
}

# 현재 위치
longitude = 127.02104432021666 
latitude = 37.51423739733134

for i in range(len(parkingzone)):# row
    payload = {
        "startX": longitude,
        "startY": latitude,
        "endX": parkingzone.loc[i, "Longitude"], #longitude
        "endY": parkingzone.loc[i, "Latitude"], # 
        "startName": "현위치",
        "endName": "주차장"
    }

    res = requests.post(url, json=payload, headers=headers)

    jsonObj = json.loads(res.text)

    if jsonObj["features"][0]["properties"]["totalTime"] < time:
        nearest = jsonObj
        # nearestX = parkingzone.loc[i, "Longitude"]
        # nearestY = parkingzone.loc[i, "Latitude"]
        # address = parkingzone.loc[i, "Address"]

print(nearest)
resultData = nearest["features"]

{'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [127.02102140621453, 37.51423096958477]}, 'properties': {'totalDistance': 386, 'totalTime': 368, 'index': 0, 'pointIndex': 0, 'name': '', 'description': '60m 이동', 'direction': '', 'nearPoiName': '', 'nearPoiX': '0.0', 'nearPoiY': '0.0', 'intersectionName': '', 'facilityType': '11', 'facilityName': '', 'turnType': 200, 'pointType': 'SP'}}, {'type': 'Feature', 'geometry': {'type': 'LineString', 'coordinates': [[127.02102140621453, 37.51423096958477], [127.02085751739446, 37.51475590564801]]}, 'properties': {'index': 1, 'lineIndex': 0, 'name': '', 'description': ', 60m', 'distance': 60, 'time': 46, 'roadType': 0, 'categoryRoadType': 0, 'facilityType': '11', 'facilityName': ''}}, {'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [127.02085751739446, 37.51475590564801]}, 'properties': {'index': 2, 'pointIndex': 1, 'name': '하영전자', 'description': '하영전자 에서 좌회전 후 52m 이동 ', 

In [ ]:
type(nearest)

dict

In [ ]:
resultData = jsonObj["features"]
print(resultData)

[{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [127.02102140621453, 37.51423096958477]}, 'properties': {'totalDistance': 386, 'totalTime': 368, 'index': 0, 'pointIndex': 0, 'name': '', 'description': '60m 이동', 'direction': '', 'nearPoiName': '', 'nearPoiX': '0.0', 'nearPoiY': '0.0', 'intersectionName': '', 'facilityType': '11', 'facilityName': '', 'turnType': 200, 'pointType': 'SP'}}, {'type': 'Feature', 'geometry': {'type': 'LineString', 'coordinates': [[127.02102140621453, 37.51423096958477], [127.02085751739446, 37.51475590564801]]}, 'properties': {'index': 1, 'lineIndex': 0, 'name': '', 'description': ', 60m', 'distance': 60, 'time': 46, 'roadType': 0, 'categoryRoadType': 0, 'facilityType': '11', 'facilityName': ''}}, {'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [127.02085751739446, 37.51475590564801]}, 'properties': {'index': 2, 'pointIndex': 1, 'name': '하영전자', 'description': '하영전자 에서 좌회전 후 52m 이동 ', 'direction': '', 'nearPoiName': '하영전자', 'n

In [ ]:
totalTime = resultData[0]["properties"]["totalTime"] # 소요 시간 (초 단위)

print(f"{totalTime} sec")
print(f"{totalTime//60}m {totalTime%60}s")

368 sec
6m 8s


In [ ]:
totalDistance = resultData[0]["properties"]["totalDistance"] # 거리 (m 단위)

print(f"{totalDistance}m")
print(f"{totalDistance//1000}km {totalDistance%1000}m")

4415m
4km 415m


In [ ]:
for data in resultData:
    print(data["geometry"])

{'type': 'Point', 'coordinates': [127.02102140621453, 37.51423096958477]}
{'type': 'LineString', 'coordinates': [[127.02102140621453, 37.51423096958477], [127.02085751739446, 37.51475590564801]]}
{'type': 'Point', 'coordinates': [127.02085751739446, 37.51475590564801]}
{'type': 'LineString', 'coordinates': [[127.02085751739446, 37.51475590564801], [127.02028812809073, 37.514639242314516]]}
{'type': 'Point', 'coordinates': [127.02028812809073, 37.514639242314516]}
{'type': 'LineString', 'coordinates': [[127.02028812809073, 37.514639242314516], [127.02000479430679, 37.51556690721547], [127.0199936831857, 37.51560301393165], [127.01998257230034, 37.51563078828269], [127.01996312758305, 37.51570300176491], [127.01997145913622, 37.515739108830104], [127.0199797905322, 37.51578077080539], [127.02000478660608, 37.515839097810115], [127.02002422794449, 37.515886314894935], [127.0200575562787, 37.51595575186942]]}
{'type': 'Point', 'coordinates': [127.0200575562787, 37.51595575186942]}
{'type':